In [4]:
library(ggplot2)
suppressPackageStartupMessages(library(extraDistr))
suppressPackageStartupMessages(library(distr))
source("./simple.R")
source("./simple_utils.R")
set.seed(2025)


### Q1: efficacy of vaccines

In [ ]:
# 2
BetaMP = function(mean, precision){
  Beta(mean*precision, (1-mean)*precision)
}

ppl_fn = function() {
    mu_e = simulate(Unif(0,1))
    lam_e = simulate(Exp(rate = 0.01))
    mu_p = simulate(BetaMP(0.1, 15))
    lam_p = simulate(Exp(rate = 0.01))
    e = simulate(BetaMP(mu_e, lam_e))
    p = simulate(BetaMP(mu_p, lam_p))
    observe(realization = 162,distribution = Binom(p, 18325))

}

In [2]:
# 3
vaccines = read.csv("./vaccines.csv")
vaccines$groupSizes = as.double(vaccines$groupSizes) # needed due to bug in Binom code